In [64]:
# Import necessary libraries
import pandas as pd
import nltk
import seaborn as sns
import plotly.express as px

# Import machine learning and text processing libraries
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Import stopwords from NLTK
from nltk.corpus import stopwords

In [65]:
# 1. Charger les données
try:
    df = pd.read_csv('points_marquants/points_marquants_24_clean.csv', sep=';')
    descriptions = df['commentaire'].dropna().tolist()
except FileNotFoundError:
    print("Erreur: Le fichier n'a pas été trouvé.")
    exit()
except KeyError:
    print("Erreur: La colonne 'REDACTION' ou la feuille spécifiée n'existe pas.")
    exit()

if not descriptions:
    print("Aucune description de panne à analyser.")
    exit()

In [66]:
# 2. Vectorisation du texte avec TF-IDF
# Télécharger les stopwords français si ce n'est pas déjà fait
nltk.download('stopwords')
french_stop_words = stopwords.words('french')

vectorizer = TfidfVectorizer(stop_words=french_stop_words)  # Utilisation des stopwords français
tfidf_matrix = vectorizer.fit_transform(descriptions)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\VBO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [67]:
# 3. Application de l'algorithme de clustering K-means
# Déterminer le nombre optimal de clusters (méthode du coude ou silhouette)
# Ici, nous allons essayer une plage de nombres de clusters
range_n_clusters = range(2, 20)
silhouette_scores = []

# Create a DataFrame to store the silhouette scores for each number of clusters
silhouette_df = pd.DataFrame(columns=['n_clusters', 'silhouette_score'])

for n_clusters in range_n_clusters:
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(tfidf_matrix)
    silhouette_avg = silhouette_score(tfidf_matrix, cluster_labels)
    silhouette_df = pd.concat([silhouette_df, pd.DataFrame({'n_clusters': [n_clusters], 'silhouette_score': [silhouette_avg]})], ignore_index=True)
    # print(f"Silhouette score pour {n_clusters} clusters: {silhouette_avg:.4f}")
    
print(silhouette_df)

C:\Users\VBO\AppData\Local\Temp\ipykernel_22876\729130300.py:14: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



   n_clusters  silhouette_score
0           2          0.046244
1           3          0.053553
2           4          0.062420
3           5          0.064351
4           6          0.076843
5           7          0.080166
6           8          0.095742
7           9          0.100616
8          10          0.106698
9          11          0.103305
10         12          0.111018
11         13          0.124507
12         14          0.120908
13         15          0.125747
14         16          0.135202
15         17          0.142121
16         18          0.124166
17         19          0.139871


In [68]:
# 4. Choisir le nombre optimal de clusters
n_clusters_optimal = 17
kmeans_optimal = KMeans(n_clusters=n_clusters_optimal, random_state=42, n_init=10)
clusters = kmeans_optimal.fit_predict(tfidf_matrix)

In [69]:
# 5. Analyse des clusters
results_df = pd.DataFrame({'commentaire': descriptions, 'Cluster': clusters})

# Transform the clustering results into a DataFrame for better readability
cluster_summary = []

for i in range(n_clusters_optimal):
    cluster_descriptions = results_df[results_df['Cluster'] == i]['commentaire'].tolist()
    cluster_summary.append({
        'Cluster': i,
        'Nombre de descriptions': len(cluster_descriptions),
        'Exemples de descriptions': cluster_descriptions[:10]  # Limiter à 10 exemples
    })

cluster_summary_df = pd.DataFrame(cluster_summary)
print(cluster_summary_df)

    Cluster  Nombre de descriptions  \
0         0                      18   
1         1                      11   
2         2                      34   
3         3                      13   
4         4                      13   
5         5                       9   
6         6                      15   
7         7                      14   
8         8                      15   
9         9                      11   
10       10                      18   
11       11                      51   
12       12                       6   
13       13                       4   
14       14                       7   
15       15                       9   
16       16                       8   

                             Exemples de descriptions  
0   [Suite vandalisme 6 Marches cassées + plaque p...  
1   [infiltration eau de pluie - Asc HS, Arret fui...  
2   [Problemes moteur + defauts surchauffe \n+ pb ...  
3   [ Câble limiteur de vitesse HS + câble command...  
4   [Visite RATP,

In [70]:
# Extract feature names from the vectorizer
terms = vectorizer.get_feature_names_out()

# Compute order_centroids from the cluster centers
order_centroids = kmeans_optimal.cluster_centers_.argsort()[:, ::-1]

# Create a DataFrame to store the keywords for each cluster
keywords_df = pd.DataFrame({
    'Cluster': range(n_clusters_optimal),
    'Top Keywords': [', '.join([terms[ind] for ind in order_centroids[i, :10]]) for i in range(n_clusters_optimal)]
})

print(keywords_df)


    Cluster                                       Top Keywords
0         0  peigne, plaque, defaut, porte, hs, contact, dé...
1         1  eau, infiltration, asc, masses, disjonctions, ...
2         2  probleme, moteur, réglages, surchauffe, disjon...
3         3  frein, bobine, hs, secour, relais, principal, ...
4         4  visite, ratp, jeux, marches, guide, brosses, r...
5         5  dp, problemes, frein, hs, distributeur, cellul...
6         6  ligne, suite, coupure, sécurité, alimentation,...
7         7  marche, galets, affaissement, marches, hs, cha...
8         8  arbre, roulements, entrainement, hs, entraînem...
9         9  poulie, hs, vandalisme, suite, remplacement, v...
10       10  main, courante, remplacement, gauche, hs, chai...
11       11  hs, porte, cabine, haut, contact, coulisseau, ...
12       12  reglages, portes, coulisseaux, parachute, hs, ...
13       13  opérateur, porte, hs, problèmes, supports, fix...
14       14  empilement, marches, bas, entrainement, ar

In [71]:
# Score de silhouette pour chaque nombre de clusters
fig = px.line(
    silhouette_df,
    x='n_clusters',
    y='silhouette_score',
    labels={'n_clusters': 'Nombre de clusters', 'silhouette_score': 'Score de Silhouette'},
    title='Score de Silhouette en fonction du nombre de clusters'
)
fig.update_traces(mode='lines+markers')
fig.update_layout(template='plotly_white')
fig.show()

# Distribution des descriptions de pannes par cluster (Pie chart)
cluster_counts = results_df['Cluster'].value_counts().sort_index()
fig = px.pie(
    values=cluster_counts.values,
    names=[f'Cluster {i}' for i in range(n_clusters_optimal)],
    title='Distribution des descriptions de pannes par cluster',
    hole=0.3
)
fig.show()

# Nombre de descriptions de pannes par cluster (Bar chart)
fig = px.bar(
    x=cluster_counts.index,
    y=cluster_counts.values,
    labels={'x': 'Cluster', 'y': 'Nombre de descriptions'},
    title='Nombre de descriptions de pannes par cluster'
)
fig.update_layout(template='plotly_white')
fig.show()

In [ ]:
# Initialiser PCA avec 2 composantes
pca = PCA(n_components=2)

# Calculer les résultats PCA
pca_result = pca.fit_transform(tfidf_matrix.toarray())

# Ajouter les résultats PCA au DataFrame
results_df['PCA Comp 1'] = pca_result[:, 0]
results_df['PCA Comp 2'] = pca_result[:, 1]

# Visualisation des clusters avec PCA (2 composantes)
fig_pca = px.scatter(
    results_df,
    x='PCA Comp 1',
    y='PCA Comp 2',
    color='Cluster',
    title='Visualisation des clusters avec PCA (2 composantes)',
    labels={'PCA Comp 1': 'PCA Composante 1', 'PCA Comp 2': 'PCA Composante 2'},
    color_continuous_scale='Viridis'
)
fig_pca.update_layout(template='plotly_white')
fig_pca.show()

# Initialiser t-SNE avec 2 composantes
tsne = TSNE(n_components=2, random_state=42)

# Calculer les résultats t-SNE
tsne_result = tsne.fit_transform(tfidf_matrix.toarray())

# Ajouter les résultats t-SNE au DataFrame
results_df['TSNE Comp 1'] = tsne_result[:, 0]
results_df['TSNE Comp 2'] = tsne_result[:, 1]

# Visualisation des clusters avec t-SNE (2 composantes)
fig_tsne = px.scatter(
    results_df,
    x='TSNE Comp 1',
    y='TSNE Comp 2',
    color='Cluster',
    title='Visualisation des clusters avec t-SNE (2 composantes)',
    labels={'TSNE Comp 1': 't-SNE Composante 1', 'TSNE Comp 2': 't-SNE Composante 2'},
    color_continuous_scale='Viridis'
)
fig_tsne.update_layout(template='plotly_white')
fig_tsne.show()